# UTILITIES

## SHELL OUTPUT

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## MODULES IMPORT

In [ ]:
import os
import tensorflow as tf
import numpy as np

## GOOGLE DRIVE

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## KAGGLE API

Kaggle API installation. We use it for download the dataset and also upload the submission at the end.

In [ ]:
!pip install -q kaggle
!pip install --upgrade --force-reinstall --no-deps kaggle
!mkdir -p ~/.kaggle
!cp ./drive/MyDrive/AN2DL-competitions/HW1/kaggle.json ~/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json
!kaggle competitions download -c artificial-neural-networks-and-deep-learning-2020

## UNZIP DATA 

In [ ]:
import zipfile
if not os.path.exists('./MaskDataset'):
  with zipfile.ZipFile('./artificial-neural-networks-and-deep-learning-2020.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/')

# PARAMS

In [ ]:
# Get current working directory
cwd = os.getcwd()
# Get Dataset directory
dataset_dir = os.path.join(cwd, 'MaskDataset')
# Image size
IMG_H = 256
IMG_W = 256
# Number of classes
NUM_CLASSES = 3
# Batch Size
BS = 32
# Transfer Learning
TL = True
# Learning Rate
LR = 1e-4
# Number of epochs
EPOCHS = 150
# Seed
SEED = 1234
# Early Stopping
ES = True
PATIENCE = 10

# GENERATORS

## IMAGE DATA GENERATOR

Data augmentation with all the transformations we retained correct. From "train_data_gen" we will obtain also the validation generator (validatio_split = 0.2).

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import preprocess_input 

if TL:
  train_data_gen = ImageDataGenerator(zoom_range=0.1,
                                      horizontal_flip=True,
                                      rotation_range=20,
                                      brightness_range=[0.6, 1.4], 
                                      fill_mode="nearest",
                                      cval=0,
                                      validation_split=0.20,
                                      preprocessing_function=preprocess_input)

else:
  train_data_gen = ImageDataGenerator(zoom_range=0.1,
                                      horizontal_flip=True,
                                      rotation_range=20,
                                      brightness_range=[0.6, 1.4], 
                                      fill_mode="nearest",
                                      cval=0,
                                      validation_split=0.20,
                                      rescale=1./255)

test_data_gen = ImageDataGenerator(rescale=1./255)

## FLOW FROM DATAFRAME

Starting from "train_data_gen" this code creates "train_gen" and "valid_gen" by the use of the attributes "subset". For the test set, due to the fact that the images do not already have classes we used the option "class_mode = None".

In [ ]:
import json
import pandas as pd
from sklearn.utils import shuffle 

with open(os.path.join(dataset_dir,"train_gt.json")) as f:
  dic = json.load(f)

# we can't use only "dic" because the classes can't be int but strings. So this code casts them and creates the dataframe.
df = pd.DataFrame({k:str(v) for k, v in dic.items()}.items())
df.rename(columns = {0:"filename", 1:"class"}, inplace = True)
df = shuffle(df)

training_dir = os.path.join(dataset_dir, 'training') 

train_gen = train_data_gen.flow_from_dataframe(dataframe=df,
                                               directory=training_dir,
                                               x_col="filename",
                                               y_col="class",
                                               batch_size=BS,
                                               target_size=(IMG_H, IMG_W),
                                               shuffle=True,
                                               class_mode='categorical',
                                               subset="training",
                                               seed=SEED)

valid_gen = train_data_gen.flow_from_dataframe(dataframe=df,
                                               directory=training_dir,
                                               x_col="filename",
                                               y_col="class",
                                               batch_size=BS,
                                               target_size=(IMG_H, IMG_W),
                                               shuffle=True,
                                               class_mode='categorical',
                                               subset="validation",
                                               seed=SEED)

test_dir = os.path.join(dataset_dir, 'test')
test_df = pd.DataFrame(os.listdir(test_dir))
test_df.rename(columns = {0:"filename"}, inplace = True)
test_gen = test_data_gen.flow_from_dataframe(dataframe=test_df,
                                             directory=test_dir,
                                             x_col="filename",
                                             batch_size=1, 
                                             target_size=(IMG_H, IMG_W),
                                             class_mode=None,
                                             shuffle=False,
                                             seed=SEED)

# MODEL

## CALLBACKS

In [ ]:
from datetime import datetime

cwd = '/content/drive/MyDrive/AN2DL-competitions/HW1'

experiments_dir = os.path.join(cwd, 'experiments-VGG16')
if not os.path.exists(experiments_dir):
    os.makedirs(experiments_dir)

now = datetime.now().strftime('%b%d_%H-%M-%S')

model_name = 'Proj1'

proj_dir = os.path.join(experiments_dir, model_name + '_' + str(now))
if not os.path.exists(proj_dir):
    os.makedirs(proj_dir)
    
callbacks = []

### Model Checkpoint

This code save only the best solution. With the option "mode = max", the best is the one with the higher validation accuracy.

In [ ]:
ckpt_dir = os.path.join(proj_dir, 'checkpoints')
if not os.path.exists(ckpt_dir):
    os.makedirs(ckpt_dir)

ckpt_callback = tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(ckpt_dir, 'cp.ckpt'),
                                                   save_weights_only=True,
                                                   save_best_only=True,
                                                   mode='max') 
callbacks.append(ckpt_callback)

### Tensorboard

In [ ]:
tb_dir = os.path.join(proj_dir, 'tensorboard-logs')
if not os.path.exists(tb_dir):
    os.makedirs(tb_dir)
    
# By default shows losses and metrics for both training and validation
tb_callback = tf.keras.callbacks.TensorBoard(log_dir=tb_dir,
                                             profile_batch=0,
                                             histogram_freq=1)
callbacks.append(tb_callback)

### Early Stopping

In [ ]:
if ES:
    es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=PATIENCE)
    callbacks.append(es_callback)

## DEFINITION

### Transfer learning

In [ ]:
if TL:
  vgg = tf.keras.applications.VGG16(weights='imagenet', 
                                           include_top=False, 
                                           input_shape=(IMG_H, IMG_W, 3))
  
vgg.summary()

In [ ]:
# this select how many layers to freeze

if TL:

  finetuning = True
  all_trainable = False

  if all_trainable:
    vgg.trainable = True
  elif finetuning:
    freeze_until = 15
    for layer in vgg.layers[:freeze_until]:
          layer.trainable = False
  else:
    vgg.trainable = False

In [ ]:
# we used the following list for create a file with the main 
# parameteres of the NN (see later "params description") 
neurons_per_layer = []
neurons_per_layer.append(128)
neurons_per_layer.append(128)


model = tf.keras.Sequential()
model.add(vgg)
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(units=neurons_per_layer[0], activation='relu'))
model.add(tf.keras.layers.Dense(units=neurons_per_layer[1], activation='relu'))

model.add(tf.keras.layers.Dense(units=NUM_CLASSES, activation='softmax'))

model.summary()

### Custom model

In [ ]:
if not TL:
  
  start_f = 8
  depth = 5

  model = tf.keras.Sequential()

  # Features extraction
  for i in range(depth):

    if i == 0:
        input_shape = [IMG_H, IMG_W, 3]
    else:
        input_shape=[None]

    # Conv block: Conv2D -> Activation -> Pooling
    model.add(tf.keras.layers.Conv2D(filters=start_f, 
                                     kernel_size=(3, 3),
                                     strides=(1, 1),
                                     padding='same',
                                     input_shape=input_shape))
    model.add(tf.keras.layers.ReLU())
    model.add(tf.keras.layers.MaxPool2D(pool_size=(2, 2)))

    start_f *= 2
      
  # Classifier
  model.add(tf.keras.layers.Flatten())
  model.add(tf.keras.layers.Dense(units=512, activation='relu'))
  model.add(tf.keras.layers.Dense(units=NUM_CLASSES, activation='softmax'))

  model.summary()

## OPTIMIZATION PARAMS

In [ ]:
# Loss
loss = tf.keras.losses.CategoricalCrossentropy()

# Optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=LR)

# Metrics
metrics = ['accuracy']

# Set random seed
tf.random.set_seed(SEED)

# Compile Model
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

# TRAINING

## PARAMS DESCRIPTION

This code creates a file in the actual project folder for remebering us the main parameters of the NN. In this way we could immediately distinguish one experiment from the others.

In [ ]:
f = open(os.path.join(proj_dir, "params.txt"), "w+")
f.write('IMG = ' + str(IMG_H) + ' x ' + str(IMG_H) + '\n')
f.write('BS = ' + str(BS) + '\n')
f.write('LR = ' + str(LR) + '\n')
f.write('EPOCHS = ' + str(EPOCHS) + '\n')
f.write('SEED = ' + str(SEED) + '\n')
f.write('ES = ' + str(ES))
if ES:
  f.write(' with PETIENCE = ' + str(PATIENCE) + '\n')
else:
  f.write('\n')
f.write('TL (VGG) = ' + str(TL) + '\n')
if finetuning and not all_trainable:
  f.write('FT = ' + str(finetuning) + ' with weights freezed until ' + str(freeze_until) + '\n')
elif all_trainable:
  f.write('All trainable\n')
for i in range(len(neurons_per_layer)):
  f.write('Layer ' + str(i) + ' with ' + str(neurons_per_layer[i]) + ' units\n')
for i in range(len(dropout_per_layer)):
  f.write('Layer ' + str(i) + ' with DROPOUT = ' + str(dropout_per_layer[i]) + ' \n')
f.close()

## TENSORBOARD

In [ ]:
%load_ext tensorboard
%tensorboard --logdir /content/drive/MyDrive/AN2DL-competitions/HW1/experiments-VGG16/ --port 6009

## FIT METHOD

In [ ]:
model.fit(train_gen,
          epochs=EPOCHS,
          steps_per_epoch=len(train_gen),
          validation_data=valid_gen,
          validation_steps=len(valid_gen), 
          callbacks=callbacks)

# CSV FOR SUBMISSION

## MODEL RE-DEFINITION

In [ ]:
del model

neurons_per_layer = []
neurons_per_layer.append(128)
neurons_per_layer.append(128)


model = tf.keras.Sequential()
model.add(vgg)
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(units=neurons_per_layer[0], activation='relu'))
model.add(tf.keras.layers.Dense(units=neurons_per_layer[1], activation='relu'))

model.add(tf.keras.layers.Dense(units=NUM_CLASSES, activation='softmax'))

model.summary()

In [ ]:
model.load_weights(tf.train.latest_checkpoint(ckpt_dir))

## CSV FUNCTION

In [ ]:
def create_csv(results, results_dir='./'):

    csv_fname = 'results.csv'

    with open(os.path.join(results_dir, csv_fname), 'w') as f:

        f.write('Id,Category\n')

        for key, value in results.items():
            f.write(key + ',' + str(value) + '\n')

## RESULTS & SUBMISSION

Exploiting the previous function this code creates the CSV file and submits it on Kaggle.

In [ ]:
predicted_class = tf.argmax(model.predict(test_gen), 1)

i=0
results={}
images_name = os.listdir(test_dir)
images_name.sort()

for f in images_name:
  results[f] =  predicted_class.numpy().tolist()[i]
  i+=1

create_csv(results)

!kaggle competitions submit -c artificial-neural-networks-and-deep-learning-2020 -f ./results.csv -m "Submission"